# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [37]:
# Dependencies and Setup

import hvplot.pandas
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import cartopy
import geoviews
import json

# Import API key
from api_keys import geoapify_key
import warnings
warnings.filterwarnings("ignore")

In [38]:
from bokeh.core.validation import silence
from bokeh.core.validation.warnings import FIXED_SIZING_MODE
silence(FIXED_SIZING_MODE, True)

## Code from Stack Overflow suggesting this workaround for 'sizing' mode warnings
## https://stackoverflow.com/questions/56758246/after-updating-to-bokeh-1-2-i-am-experiencing-several-problems-running-my-bokeh

{Warning(code=1000, name='MISSING_RENDERERS', description='Plot has no renderers'),
 Warning(code=1002, name='EMPTY_LAYOUT', description='Layout has no children'),
 Warning(code=1005, name='FIXED_SIZING_MODE', description="'fixed' sizing mode requires width and height to be set")}

In [39]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,13.12,68,23,4.93,SH,1701129771
1,1,kodiak,57.7900,-152.4072,4.73,45,0,3.60,US,1701129772
2,2,utrik,11.2278,169.8474,27.70,74,74,8.54,MH,1701129773
3,3,yellowknife,62.4560,-114.3525,-6.71,100,100,4.63,CA,1701129688
4,4,punta arenas,-53.1500,-70.9167,12.06,40,0,5.14,CL,1701129546


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [40]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    color = "City"
)
hvplot.save(map_plot_1, '../output_data/AllCities.html')
# Display the map
map_plot_1


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [41]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df[ (city_data_df["Max Temp"] >= 22) &  (city_data_df["Humidity"] <= 60) ]
clean_ideal_city_df = ideal_df[ ideal_df["Cloudiness"] == 0 ] 

# Drop any rows with null values
clean_ideal_data_df = clean_ideal_city_df.dropna()

# Display sample data
clean_ideal_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
44,44,selebi-phikwe,-21.9790,27.8498,23.40,39,0,2.26,BW,1701129804
62,62,leyva solano,25.6667,-108.6417,28.31,44,0,2.86,MX,1701129825
329,329,rosario,-32.9468,-60.6393,29.97,48,0,3.09,AR,1701129943
504,504,ormara,25.2088,64.6357,24.17,47,0,0.89,PK,1701130142


### Step 3: Create a new DataFrame called `hotel_df`.

In [42]:
hotel_df = clean_ideal_data_df[['City','Country','Lat','Lng','Humidity']].copy()
hotel_df['Hotel Name'] = ""
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
44,selebi-phikwe,BW,-21.9790,27.8498,39,
62,leyva solano,MX,25.6667,-108.6417,44,
329,rosario,AR,-32.9468,-60.6393,48,
504,ormara,PK,25.2088,64.6357,47,


In [43]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_ideal_data_df[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Reset Index
newindex = np.arange(0,len(hotel_df))
hotel_df = hotel_df.set_index(newindex)

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,selebi-phikwe,BW,-21.9790,27.8498,39,
1,leyva solano,MX,25.6667,-108.6417,44,
2,rosario,AR,-32.9468,-60.6393,48,
3,ormara,PK,25.2088,64.6357,47,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [44]:
# Set parameters to search for a hotel

radius = 10000
categories = 'accommodation.hotel'
limit = 1
params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    city_lat = row.loc['Lat']
    city_lng = row.loc['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    filter = f"circle:{city_lng},{city_lat},{radius}"
    bias = f"proximity:{city_lng},{city_lat}"
    params["filter"] = filter
    params["bias"] = bias
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
selebi-phikwe - nearest hotel: Cresta Bosele Hotel
leyva solano - nearest hotel: No hotel found
rosario - nearest hotel: Fontana Suites
ormara - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
0,selebi-phikwe,BW,-21.9790,27.8498,39,Cresta Bosele Hotel
1,leyva solano,MX,25.6667,-108.6417,44,No hotel found
2,rosario,AR,-32.9468,-60.6393,48,Fontana Suites
3,ormara,PK,25.2088,64.6357,47,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [45]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    color = "City",
    hover_cols = ['City','Humidity','Hotel Name','Country'],

)
hvplot.save(map_plot_2, '../output_data/Hotels.html')

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)